In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
import time
from datetime import datetime, timedelta
import shutil
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import statsmodels.tsa.api as tsa
import torch
from pandarallel import pandarallel
import seaborn as sns
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.preprocessing import StandardScaler



In [2]:
X_order = np.load('tmp/sorted_order.npy')
X_page = np.load('tmp/sorted_page_day60.npy')
X_metric = np.load('tmp/sorted_metric.npy')
y = np.load('tmp/sorted_label.npy')

In [4]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--d_model', type=int, default=128, help='dimension of model')
parser.add_argument('--e_layers', type=int, default=3, help='num of encoder layers')
parser.add_argument('--seq_len', type=int, default=60, help='input sequence length')
parser.add_argument('--dropout', type=float, default=0.3, help='dropout')
parser.add_argument('--n_head', type=int, default=8, help='num of heads')
parser.add_argument('--ffn_hidden', type=int, default=512, help='dimension of hidden layer')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=20, help='training epochs')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--val_rate', type=float, default=0.1, help='validation rate')
parser.add_argument('--n_components', type=int, default=450)
parser.add_argument('--d_seq', type=int, default=18)
parser.add_argument('--d_light_in', type=int, default=32)
parser.add_argument('--d_emb', type=int, default=2)
parser.add_argument('--d_light_tar', type=int, default=1)

args = parser.parse_args(args = [])

In [ ]:
%run models/ICCP.py
k = 10
from sklearn.model_selection import StratifiedKFold
f1_list = []
auc_list = []
sfolder = StratifiedKFold(n_splits=k, shuffle=True)
fold_count = 1
for train_index, test_index in sfolder.split(np.arange(y.shape[0]), y):
    print("--------------------- fold {} ----------------------".format(fold_count))
    fold_count += 1
    X_train_order = []
    X_test_order = []
    X_train_page = []
    X_test_page = []
    X_train_metric = []
    X_test_metric = []
    y_train = []
    y_test = []


    for id in train_index:
        X_train_order.append(X_order[id])
        X_train_page.append(X_page[id])
        X_train_metric.append(X_metric[id])
        y_train.append(y[id])

    for id in test_index:
        X_test_order.append(X_order[id])
        X_test_page.append(X_page[id])
        X_test_metric.append(X_metric[id])
        y_test.append(y[id])

    X_train_order = np.array(X_train_order)
    X_train_page = np.array(X_train_page)
    X_train_metric = np.array(X_train_metric)
    X_test_order = np.array(X_test_order)
    X_test_page = np.array(X_test_page)
    X_test_metric = np.array(X_test_metric)

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    scale = False
    if scale:
        from sklearn.preprocessing import MinMaxScaler
        min_max_scaler = MinMaxScaler()
        X_train_order = min_max_scaler.fit_transform(X_train_order)
        X_test_order = min_max_scaler.transform(X_test_order)
    
    
    X_train_both = np.hstack((X_train_order, X_train_page.reshape(X_train_page.shape[0], -1)))
    X_test_both = np.hstack((X_test_order, X_test_page.reshape(X_test_page.shape[0], -1)))
    
    # --------------- ICCP Test ---------------------------------------------------
    
    args.verbose = False
    exp = Exp(args)
    exp.train(X_train_both, X_train_metric, y_train)
    f1, auc = exp.get_metric(X_test_both, X_test_metric, y_test)
    exp.predict(X_test_both, X_test_metric, y_test)
    f1_list.append(maxf1)
    auc_list.append(maxauc)
        
    

In [ ]:
f1_list

In [ ]:
auc_list

In [ ]:
np.array(f1_list).mean()

In [ ]:
np.array(auc_list).mean()